# Retrieval

In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain.llms import OpenAI

llm = OpenAI(model='gpt-3.5-turbo-instruct')

C:\Users\annaf\AppData\Local\Temp\ipykernel_7280\1267886439.py:6: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model='gpt-3.5-turbo-instruct')


In [2]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

caminhos = [
    'assets/Explorando o Universo das IAs com Hugging Face.pdf',
    'assets/Explorando a API da OpenAI.pdf',
    'assets/Explorando a API da OpenAI.pdf'
]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())
    
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=['\n\n', '\n', '.', ' ', '']
)

documents = recur_split.split_documents(paginas)

## Modificando metadata

In [4]:
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('assets/', '')
    doc.metadata['doc_id'] = i

In [8]:
documents[0].metadata

{'source': 'Explorando o Universo das IAs com Hugging Face.pdf',
 'page': 0,
 'doc_id': 0}

In [9]:
# __import__('pysqlite3)
# import sys
# sys.modules['sqlite3] = sys.modules.pop('pysqlite3')

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings_model = OpenAIEmbeddings()

### Criando VectorStore

In [10]:
diretorio = 'docs/chroma_retrieval'

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

### Semantic Search | Busca Semântica

### Limitações do Semantic Search

#### Trechos com alta similaridade e pouca informação adicional

In [11]:
pergunta = 'O que é a OpenAI?'

docs = vectorstore.similarity_search(pergunta, k=3)

for doc in docs:
    print(f'{doc.page_content}\n')
    print(f'========== {doc.metadata}\n\n')

Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira como lidamos com informações e realiza tarefas que antes pareciam

========== {'doc_id': 283, 'page': 4, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira

#### Busca em arquivos específicos não funciona corretamente

In [12]:
pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT??'

docs = vectorstore.similarity_search(pergunta, k=3)

for doc in docs:
    print(f'{doc.page_content}\n')
    print(f'========== {doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes modelos estão classificados na tarefa de geração de texto (text
generation) no Hugging Face:
Figure 16: Página principal da tarefatext-generation do Hugging Face.
Vamos utilizar o modelo Felladrin/Llama-68M-Chat-v1, disponível no link a seguir:

========== {'doc_id': 86, 'page': 29, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando a API da OpenAI
A OpenAI
Nosso objetivo é explorar a API da OpenAI, a maior desenvolvedora de ferramentas de inteligência
artificial do mundo na atualidade. O modelo ChatGPT é apenas um dos vários modelos criados pela
empresa nos últimos anos. A seguir, apresentamos alguns dos principais modelos e suas funções:
• GPT-4: Trata-se de um modelo de linguagem de grande escala (LLM) multimodal, que aceita


### MMR

In [13]:
pergunta = 'O que é a OpenAI?'

docs = vectorstore.max_marginal_relevance_search(pergunta, k=3, fetch_k=10)

for doc in docs:
    print(f'{doc.page_content}\n')
    print(f'========== {doc.metadata}\n\n')

Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira como lidamos com informações e realiza tarefas que antes pareciam

========== {'doc_id': 283, 'page': 4, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
Guias
Também está disponível diversos guias de boas práticas recomendadas ao utilizar os modelos da
OpenAI.
Asimov Academy 9

========== {'doc_id': 299, 'page': 9, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
A OpenAI
Nosso objetivo é explorar a API da OpenAI, a maior desenvolvedora de ferramentas de inteligência
artificial do mundo na atualidade. O modelo ChatGPT é 

### Filtragem

In [16]:
pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT?'

docs = vectorstore.similarity_search(
    pergunta, 
    k=3,
    filter={'source': {'$in': ['Explorando o Universo das IAs com Hugging Face.pdf']}})
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes modelos estão classificados na tarefa de geração de texto (text
generation) no Hugging Face:
Figure 16: Página principal da tarefatext-generation do Hugging Face.
Vamos utilizar o modelo Felladrin/Llama-68M-Chat-v1, disponível no link a seguir:
==========={'doc_id': 86, 'page': 29, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
01. O que é Hugging Face?
Bem-vindos ao curso de Hugging Face da Asimov Academy!
Neste curso, vamos explorar as principais utilidades da plataforma de IAHugging Face. Aprenderemos
como utilizar a plataforma ao máximo, e como incorporar as bibliotecas de Python do Hugging Face
aos nossos scripts de Python.
Afinal, o que é Hugging Face?
A Hugging Face é uma empresa qu